<a href="https://colab.research.google.com/github/LeeYuuuan/Graph-Kernel-Isomorphic-/blob/main/Graph_Kernel_for_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# add dependencies


In [20]:
%pip install grakel
import grakel
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import networkx.algorithms.isomorphism as iso
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


from sklearn.metrics import classification_report 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import make_pipeline

import seaborn as sns

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# draw graph


In [2]:
def draw_graphs(G,G1):
    pos = nx.spring_layout(G)
    pos1 = nx.spring_layout(G1) 
    plt.subplot(121)
    nx.draw(G, pos, with_labels=True, node_color='pink', node_size=500)
    plt.subplot(122)
    nx.draw(G1, pos1, with_labels=True, node_size=500)
    plt.show()


# load data

In [3]:
def GetGraphData(dataset_name):
  dataset = grakel.datasets.fetch_dataset(dataset_name, verbose=True, data_home=None, download_if_missing=True, with_classes=True, produce_labels_nodes=False, prefer_attr_nodes=False, prefer_attr_edges=False, as_graphs=False)
  G = dataset.data
  y = dataset.target
  return G, y

# wl_test for classification

In [4]:
def CreateFeatureMatrix_wl_test(vector_all):
  feature_matrix = np.zeros([len(vector_all),len(vector_all[0])])
  for i in range(len(vector_all)):
      feature_matrix[i,:] = vector_all[i]
      #feature_matrix[i,j] = IsIsomorphicDefault(graphs[i],graphs[j],is_node_labeled,is_edge_labeled)
   
  return feature_matrix


In [32]:
def Classification(dataset_name, kernel_name):
    G, y = GetGraphData(dataset_name)
    G_train, G_test, y_train, y_test = train_test_split(G, y, test_size=0.1, random_state=42)

    C_grid = (10. ** np.arange(-4,6,1) / len(G)).tolist()
    if kernel_name == "weisfeiler_lehman":

        gk = grakel.GraphKernel(kernel=[{"name": "weisfeiler_lehman", "n_iter": 5}, "subtree_wl"] ,Nystroem=20)
        K_train = gk.fit_transform(G_train)
        K_test = gk.transform(G_test)

        clf = SVC(kernel='linear')
        clf.fit(K_train, y_train)
        SVC(kernel='linear')
        C_grid = (10. ** np.arange(-4,6,1) / len(G)).tolist()
        scores = cross_val_score(clf, K_train, y_train, cv=5)
        y_pred = clf.predict(K_test)
        print(str(round(accuracy_score(y_test, y_pred)*100, 2)), "%")
      
    
    if kernel_name == "ShortestPath":
        estimator = make_pipeline(
        grakel.kernels.ShortestPath(normalize=True),
        GridSearchCV(SVC(kernel='precomputed'), dict(C=C_grid),
                    scoring='accuracy', cv=10))
        
        n_folds = 10
        cvp = cross_val_predict(estimator, G, y, cv=n_folds)
        acc = accuracy_score(y, cvp)
        print("Accuracy:", str(round(acc*100, 2)) + "%")
        print(classification_report(y,cvp))

        # sns.heatmap(cm,cmap = sns.color_palette("Blues"),annot = True

In [33]:
t0 = time.time()
print(Classification("BZR", "ShortestPath"))
print("done in %0.3fs\n" % (time.time() - t0))

Extracting dataset  BZR..
Parsing dataset  BZR..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Accuracy: 82.72%
              precision    recall  f1-score   support

          -1       0.84      0.97      0.90       319
           1       0.72      0.30      0.43        86

    accuracy                           0.83       405
   macro avg       0.78      0.64      0.66       405
weighted avg       0.81      0.83      0.80       405

None
done in 44.028s

